In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from itertools import chain
from sys import platform
from copy import deepcopy as dc

vims_wave = np.loadtxt('vims_wave.txt')

new_array = ['_0405','_0607','_0809','_1011','_1213','_1415','_1617']

if platform == 'darwin' or platform == 'win32':
    for number in new_array:
        new_string = 'df'+number+' = pd.read_pickle("c_data/data'+number+'.pkl")'
        exec(new_string)
else:
    df_list = []
    for number in new_array:
        new_string = 'df'+number+' = pd.read_csv("c_data/data'+number+'.csv")'
        exec(new_string)
        df = 'df'+number
        new_df = locals()[df]
        df_list.append(new_df)
    for df in df_list:
        for i in range(len(df['spectrum'])):
            df['spectrum'][i] = np.array([float(x) for x in df['spectrum'][i][1:-1].split(',')])

def powerlaw(x,a,b):
    return a*np.power(x,b)

def gaussian(x,a,mu,sigma):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

def flatten(some_list):
    flat_list = []
    for element in some_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

band_channels = list(chain(range(29,35),range(46,60),range(78,96),range(102,106)))

window = list(range(56,83))

def fit_line(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_fit = powerlaw(vims_wave,*my_fit)
    return final_fit

def custom_fit(dataframe):
    spectra = np.average(dataframe)
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectra[band_channels],p0=[.15,-12])
    final_spectra = spectra-powerlaw(vims_wave,*my_fit)
    return final_spectra

def single_fit(spectrum):
    my_fit,_ = curve_fit(powerlaw,vims_wave[band_channels],spectrum[band_channels],p0=[.15,-12])
    final_spectrum = spectrum-powerlaw(vims_wave,*my_fit)

def shift_ret(spectrum):
    my_fit,_ = curve_fit(gaussian,vims_wave[window],spectrum[window],p0=[.05,2.05])
    return my_fit[1]

In [3]:
spec_list = []
for number in new_array:
    my_string = 'old_spec'+number+' = df'+number+"['spectrum']"
    exec(my_string)
    new_string = 'spec'+number+' = np.array([np.abs(x) for x in old_spec'+number+'])' # Ensures no negative I/F values
    exec(new_string)
    spec = 'spec'+number
    spec_list.append(locals()[spec])

copy_list = []
for number in new_array:
    my_string = 'copy_spec'+number+' = dc(spec'+number+')'
    exec(my_string)
    copy_spec = 'copy_spec'+number
    copy_list.append(locals()[copy_spec])

total_spectrum = []
for spec in spec_list:
    for spectrum in spec:
        total_spectrum.append(np.abs(spectrum))

file_list = []
for number in new_array:
    my_string = 'file'+number+' = df'+number+"['fileid']"
    exec(my_string)
    id = 'file'+number
    file_list.append(locals()[id])

total_files = []
for year in file_list:
    for id in year:
        total_files.append(id)


total_spectrum = np.asarray(total_spectrum)
total_spectrum_2 = dc(total_spectrum)
total_files = np.asarray(total_files)
print('List 1: '+str(np.shape(total_spectrum)))
print('List 2: '+str(np.shape(total_spectrum_2))) # Confirms that DeepCopy works correctly

print('Total FileID List: '+str(np.shape(total_files)))

List 1: (12655, 256)
List 2: (12655, 256)
Total FileID List: (12655,)


In [4]:
np.unique(total_files)

array([19753, 19758, 19779, 19794, 19811, 19823, 19826, 20176, 20239,
       20243, 20295, 20447, 20502, 20561, 20776, 20803, 20812, 20814,
       20869, 21042, 21088, 21124, 21162, 21170, 21176, 21196, 21198,
       21211, 21217, 21223, 21245, 21254, 21258, 21275, 21283, 21298,
       21303, 21309, 21318, 21322, 21332, 21336, 21341, 21344, 21348,
       21356, 21360, 21377, 21385, 21419, 21422, 21430, 21431, 21441,
       21450, 21452, 21468, 21476, 21481, 21494, 21496, 21505, 21506,
       21512, 21523, 21530, 21535, 21553, 21554, 21561, 21569, 21571,
       21616, 21630, 21638, 21682, 26218, 26224, 26274, 26293, 26430,
       26441, 26466, 26508, 26547, 26605, 26645, 26669, 26693, 26724,
       26728, 26737, 26741, 26743, 26799, 26800, 26804, 26821, 26846,
       26897, 26943, 26963, 26975, 27011, 27018, 27058, 27082, 27111,
       27146, 27175, 27210, 27321, 27343, 27371, 27391, 27468, 27516,
       27650, 27675, 27681, 27751, 27757, 27810, 27864, 27873, 27880,
       27947, 27951,